In [1]:
from pathlib import Path
from sklearn.cluster import KMeans
import numpy as np
import SimpleITK as sitk

import matplotlib.pyplot as plt

from tqdm import tqdm

In [26]:
def sitk_cropp_padd_img_to_size(img, size=(256, 256, 124), padding_value=0):
    padding_filter = sitk.ConstantPadImageFilter()
    cropping_filter = sitk.CropImageFilter()
    for dim in range(len(img.GetSize())):
        size_origin = [0, 0, 0]
        size_end = [0, 0, 0]
        padding_size = size[dim] - img.GetSize()[dim]
        
        # padd image
        if padding_size>0:
            padding_size = abs(padding_size)
            size_origin[dim] = padding_size//2
            size_end[dim] = padding_size - size_origin[dim]
            # padd image from the left
            img = padding_filter.Execute(img, size_origin, size_end, padding_value)
        
        # crop the image
        elif padding_size < 0:
            padding_size = abs(padding_size)
            size_origin[dim] = padding_size//2
            size_end[dim] = padding_size - size_origin[dim]
            img = cropping_filter.Execute(img, size_origin, size_end)
    return img

In [27]:
tissues_labels = Path('/mrhome/vladyslavz/git/central-sulcus-analysis/data/brainvisa_augm/nobackup/tissues').glob('*.nii.gz')

for tissue_path in tqdm(tissues_labels, total=62):
    tissues = sitk.ReadImage(str(tissue_path))
    sulci_path = str(tissue_path).replace('tissues', 'sulci')
    sulci_img = sitk.ReadImage(sulci_path)
    sulci = sitk.GetArrayFromImage(sulci_img)
    
    tissues = sitk.GetArrayFromImage(tissues)

    # encode tissues
    tissues[tissues == 125] = 1000
    tissues[tissues == 126] = 2000
    tissues[tissues == 127] = 3000
    tissues[tissues == 128] = 4000
    
    tissue_labmap = sitk.GetImageFromArray(sulci + tissues)
    tissue_labmap.CopyInformation(sulci_img)
    
    # padd images
    if sulci_img.GetSize() != (256, 256, 124):
        tissue_labmap = sitk_padd_img_to_size(tissue_labmap)
    
    sitk.WriteImage(tissue_labmap, str(tissue_path).replace('tissues', 'sulci_tissues'))
    



100%|██████████| 62/62 [00:17<00:00,  3.50it/s]


In [7]:
tissues.shape

(124, 256, 256)

### Marking Genereation Labels

In [67]:
np.load('/mrhome/vladyslavz/git/SynthSeg/data/labels_classes_priors/generation_labels.npy')

array([  0,  14,  15,  16,  24,  72,  85, 502, 506, 507, 508, 509, 511,
       512, 514, 515, 516, 530,   2,   3,   4,   5,   7,   8,  10,  11,
        12,  13,  17,  18,  25,  26,  28,  30, 136, 137,  41,  42,  43,
        44,  46,  47,  49,  50,  51,  52,  53,  54,  57,  58,  60,  62,
       163, 164], dtype=int32)

In [68]:
# consider all possible intersections and combinations of sulci+tissue for genereation
generation_labels = np.concatenate([np.unique(sulci), np.unique(sulci)[:]+1000, np.unique(sulci)[:]+2000, np.unique(sulci)[:]+3000, np.unique(sulci)[:]+4000])
print(len(generation_labels))
print(generation_labels)
-++
np.save('/mrhome/vladyslavz/git/central-sulcus-analysis/data/brainvisa_augm/generation_labels.npy', generation_labels)

600
[   0    1    3    4    5    6    7    8    9   10   11   12   13   14
   15   16   17   18   19   20   21   22   23   25   26   28   29   30
   31   32   33   34   35   36   37   38   39   40   41   42   43   44
   45   46   48   49   50   51   52   53   54   57   58   59   60   61
   62   63   64   65   66   67   68   69   70   71   72   73   74   75
   76   77   78   79   80   81   82   83   84   85   86   87   88   89
   90   91   92   93   94   95   96   97   98   99  101  102  103  104
  105  106  107  108  109  110  111  112  113  114  115  116  117  118
  119  121  122  123  124  127  128  134 1000 1001 1003 1004 1005 1006
 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1017 1018 1019 1020
 1021 1022 1023 1025 1026 1028 1029 1030 1031 1032 1033 1034 1035 1036
 1037 1038 1039 1040 1041 1042 1043 1044 1045 1046 1048 1049 1050 1051
 1052 1053 1054 1057 1058 1059 1060 1061 1062 1063 1064 1065 1066 1067
 1068 1069 1070 1071 1072 1073 1074 1075 1076 1077 1078 1079 1080 1081
 1

In [69]:
# All labels of sulci without tissue intersection are CSF and are unsided + 0 labels
n_neutral_labels = sum(generation_labels<1000)
n_neutral_labels

120

In [76]:
WM_mask = (generation_labels < 3000) & (generation_labels >= 1000) 
GM_mask = generation_labels >= 3000

print(sum(GM_mask&WM_mask))
generation_classes = (GM_mask*2 + WM_mask*1).astype(int) # following convention of SynthSeg
np.save('/mrhome/vladyslavz/git/central-sulcus-analysis/data/brainvisa_augm/generation_classes.npy', generation_classes)

0


In [71]:
prior_means = np.load('/mrhome/vladyslavz/git/SynthSeg/data/labels_classes_priors/prior_means_t1.npy')
prior_stds = np.load('/mrhome/vladyslavz/git/SynthSeg/data/labels_classes_priors/prior_stds_t1.npy')

print(prior_means)

prior_means = prior_means[:, [0, 3, 4]]
prior_stds = prior_stds[:, [0, 3, 4]]# background,GM, WM

np.save('/mrhome/vladyslavz/git/central-sulcus-analysis/data/brainvisa_augm/prior_means.npy', prior_means)
np.save('/mrhome/vladyslavz/git/central-sulcus-analysis/data/brainvisa_augm/prior_stds.npy', prior_stds)

[[  7.          44.43852646 160.29378443  96.29526152 135.95211069
   88.15506261 129.52385575 124.05468047 151.79945792 142.53280645
  116.10473738  86.83433108  59.62853626 129.78496267 126.37696696
   18.11697145  93.90532281 117.13488728  69.91289748 113.7338777
   68.23628889  48.42668289  30.19849003  94.24268292 144.4557237 ]
 [  1.75        11.21000137  21.65584467  12.17831661  13.41123838
   10.55970441  17.68538813  15.72768241  19.51073198  16.05725779
   14.8217615   17.47509896  10.4086569   12.2351825   24.79729954
    3.8432211    9.12899967  11.66694586   9.24940558  12.41801427
   11.09552493   8.56551669   6.86072804  18.31996116  16.31849085]]


In [72]:
prior_means

array([[  7.        ,  96.29526152, 135.95211069],
       [  1.75      ,  12.17831661,  13.41123838]])

In [73]:
generation_classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [74]:
np.unique(sulci)

array([  0,   1,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  25,  26,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  45,  46,  48,  49,  50,  51,  52,  53,  54,  57,
        58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
       111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124,
       127, 128, 134], dtype=int16)

In [29]:
tissues_sulci.shape

(180, 2)

In [19]:
tissues_sulci[tissues_sulci]

array([[125, 125, 125, ..., 126, 126, 126],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=int16)

In [4]:
img_arrary = sitk.GetArrayFromImage(img)

In [5]:
pixels = img_arrary[img_arrary > 0]

In [7]:
kmeans = KMeans(n_clusters=2, random_state=0)
pixel_labels = kmeans.fit_predict(pixels.reshape(-1, 1))

/mrhome/vladyslavz/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [15]:
img_labels = np.zeros_like(img_arrary)
img_labels[img_arrary > 0] = pixel_labels + 100

In [17]:
label_img = sitk.GetImageFromArray(img_labels)
label_img.CopyInformation(sulci_map)

In [18]:
sitk.WriteImage(label_img+sulci_map, '/mrhome/vladyslavz/git/central-sulcus-analysis/data/brainvisa/ammon/t1mri/t1/ammon_labels.nii.gz')

In [13]:
sulci_map = '/mrhome/vladyslavz/git/central-sulcus-analysis/data/brainvisa/ammon/t1mri/t1/default_analysis/folds/3.3/base2018_manual/segmentation/RSulci_ammon_base2018_manual.nii.gz'

In [14]:
sulci_map = sitk.ReadImage(sulci_map)

In [ ]:
sulci_map